In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

# --- Step 1: Load Data and Initial Inspection ---
print("--- Step 1: Load Data and Initial Inspection ---")
file_path = 'WA_Fn-UseC_-HR-Employee-Attrition.csv'
df = None # Initialize df to None

try:
    # Load the dataset
    df = pd.read_csv(file_path)
    print(f"Dataset '{file_path}' loaded successfully.")

    # Display the first 5 rows
    print("\nFirst 5 rows of the raw dataset:")
    print(df.head().to_markdown(index=False, numalign="left", stralign="left"))

    # Display basic info
    print("\nInitial Dataset Info:")
    df.info()

except FileNotFoundError:
    print(f"Error: The file '{file_path}' was not found.")
    # Exit or handle error appropriately if file not found
    exit()
except Exception as e:
    print(f"An error occurred during loading: {e}")
    # Exit or handle error appropriately
    exit()

--- Step 1: Load Data and Initial Inspection ---
Dataset 'WA_Fn-UseC_-HR-Employee-Attrition.csv' loaded successfully.

First 5 rows of the raw dataset:
| Age   | Attrition   | BusinessTravel    | DailyRate   | Department             | DistanceFromHome   | Education   | EducationField   | EmployeeCount   | EmployeeNumber   | EnvironmentSatisfaction   | Gender   | HourlyRate   | JobInvolvement   | JobLevel   | JobRole               | JobSatisfaction   | MaritalStatus   | MonthlyIncome   | MonthlyRate   | NumCompaniesWorked   | Over18   | OverTime   | PercentSalaryHike   | PerformanceRating   | RelationshipSatisfaction   | StandardHours   | StockOptionLevel   | TotalWorkingYears   | TrainingTimesLastYear   | WorkLifeBalance   | YearsAtCompany   | YearsInCurrentRole   | YearsSinceLastPromotion   | YearsWithCurrManager   |
|:------|:------------|:------------------|:------------|:-----------------------|:-------------------|:------------|:-----------------|:----------------|:---------------

In [ ]:
# --- Step 2: Check for Missing Values ---
print("\n--- Step 2: Check for Missing Values ---")
missing_values = df.isnull().sum()
missing_values = missing_values[missing_values > 0]

if missing_values.empty:
    print("No missing values found.")
else:
    print("\nColumns with missing values and counts:")
    print(missing_values.to_markdown(numalign="left", stralign="left"))
    # Add handling logic here if missing values were found (e.g., imputation, dropping)


--- Step 2: Check for Missing Values ---
No missing values found.


In [ ]:
# --- Step 3: Check for Duplicate Rows ---
print("\n--- Step 3: Check for Duplicate Rows ---")
num_duplicates = df.duplicated().sum()

if num_duplicates == 0:
    print("No duplicate rows found.")
else:
    print(f"Found {num_duplicates} duplicate rows.")
    # Remove duplicates
    print("Removing duplicate rows...")
    df.drop_duplicates(inplace=True)
    df.reset_index(drop=True, inplace=True)
    print(f"Duplicates removed. New shape: {df.shape}")


--- Step 3: Check for Duplicate Rows ---
No duplicate rows found.


In [ ]:
# --- Step 4: Identify and Remove Columns with Only One Value ---
print("\n--- Step 4: Identify and Remove Single-Value Columns ---")
unique_counts = df.nunique()
single_value_cols = unique_counts[unique_counts == 1].index.tolist()

if not single_value_cols:
    print("No columns with only a single unique value found.")
else:
    print(f"\nColumns with only one value: {', '.join(single_value_cols)}")
    print(f"Dropping single-value columns...")
    df.drop(columns=single_value_cols, inplace=True)
    print(f"Columns dropped. New shape: {df.shape}")


--- Step 4: Identify and Remove Single-Value Columns ---

Columns with only one value: EmployeeCount, Over18, StandardHours
Dropping single-value columns...
Columns dropped. New shape: (1470, 32)


In [ ]:
# --- Step 5: Identify Potential Outliers (Example: MonthlyIncome) ---
print("\n--- Step 5: Identify Potential Outliers (Example: MonthlyIncome) ---")
numerical_column_to_check = 'MonthlyIncome'

if numerical_column_to_check in df.columns and pd.api.types.is_numeric_dtype(df[numerical_column_to_check]):
    Q1 = df[numerical_column_to_check].quantile(0.25)
    Q3 = df[numerical_column_to_check].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    outliers_mask = (df[numerical_column_to_check] < lower_bound) | (df[numerical_column_to_check] > upper_bound)
    num_outliers = outliers_mask.sum()

    if num_outliers == 0:
        print(f"No potential outliers found in '{numerical_column_to_check}' based on IQR.")
    else:
        print(f"Potential outliers identified in '{numerical_column_to_check}': {num_outliers}")
        print(f"  - Lower Bound: {lower_bound:.2f}, Upper Bound: {upper_bound:.2f}")
        print("  (Note: Outlier handling requires domain knowledge/specific analysis goals)")
else:
    print(f"Column '{numerical_column_to_check}' not found or is not numeric.")


--- Step 5: Identify Potential Outliers (Example: MonthlyIncome) ---
Potential outliers identified in 'MonthlyIncome': 114
  - Lower Bound: -5291.00, Upper Bound: 16581.00
  (Note: Outlier handling requires domain knowledge/specific analysis goals)


In [ ]:
# --- Step 6: Encode Categorical Variables ---
print("\n--- Step 6: Encode Categorical Variables ---")
categorical_cols = df.select_dtypes(include='object').columns.tolist()
# EmployeeNumber is an identifier, not typically encoded
# Attrition is often the target, mapped separately if needed
cols_to_encode = [col for col in categorical_cols if col not in ['Attrition', 'EmployeeNumber']]

if not cols_to_encode:
    print("No categorical columns identified for encoding (excluding Attrition).")
else:
    print(f"Encoding columns: {', '.join(cols_to_encode)}")
    # Using one-hot encoding
    df = pd.get_dummies(df, columns=cols_to_encode, drop_first=True, dtype=int)
    print("Categorical columns encoded.")
    print(f"DataFrame shape after encoding: {df.shape}")

# Map 'Attrition' if it's still an object type
if 'Attrition' in df.columns and df['Attrition'].dtype == 'object':
    print("Mapping 'Attrition' column (Yes=1, No=0)...")
    df['Attrition'] = df['Attrition'].map({'Yes': 1, 'No': 0})
    print("'Attrition' mapped.")


--- Step 6: Encode Categorical Variables ---
Encoding columns: BusinessTravel, Department, EducationField, Gender, JobRole, MaritalStatus, OverTime
Categorical columns encoded.
DataFrame shape after encoding: (1470, 46)
Mapping 'Attrition' column (Yes=1, No=0)...
'Attrition' mapped.


In [ ]:
# --- Step 7: Normalize/Scale Numerical Features (Example - Optional) ---
print("\n--- Step 7: Normalize/Scale Numerical Features (Example) ---")
# Scaling is often data/model dependent. This demonstrates Min-Max scaling.
# Decide if you need this for your specific visualization/analysis.

# Identify numerical columns suitable for scaling
numerical_cols = df.select_dtypes(include=np.number).columns.tolist()
# Define columns to exclude (identifiers, target, ordinal, one-hot encoded)
cols_to_exclude_from_scaling = ['Attrition', 'EmployeeNumber',
                                'Education', 'EnvironmentSatisfaction', 'JobInvolvement',
                                'JobLevel', 'JobSatisfaction', 'PerformanceRating',
                                'RelationshipSatisfaction', 'StockOptionLevel', 'WorkLifeBalance']
one_hot_cols = [col for col in df.columns if '_' in col or col in ['Gender_Male', 'OverTime_Yes']] # Adjust if needed
cols_to_exclude_from_scaling.extend(one_hot_cols)
cols_to_scale = [col for col in numerical_cols if col not in cols_to_exclude_from_scaling]

if not cols_to_scale:
    print("No numerical columns identified for scaling based on exclusion criteria.")
else:
    print(f"Columns identified for potential scaling: {', '.join(cols_to_scale)}")
    scaler = MinMaxScaler()
    # Example: Create a scaled copy (don't modify df unless intended)
    df_scaled_example = df.copy()
    df_scaled_example[cols_to_scale] = scaler.fit_transform(df[cols_to_scale])
    print("Example scaling applied (to df_scaled_example, not df). First 5 Age/MonthlyIncome:")
    print(df_scaled_example[['Age', 'MonthlyIncome']].head().to_markdown(index=False, numalign="left", stralign="left"))
    # To apply scaling permanently: df[cols_to_scale] = scaler.fit_transform(df[cols_to_scale])


--- Step 7: Normalize/Scale Numerical Features (Example) ---
Columns identified for potential scaling: Age, DailyRate, DistanceFromHome, HourlyRate, MonthlyIncome, MonthlyRate, NumCompaniesWorked, PercentSalaryHike, TotalWorkingYears, TrainingTimesLastYear, YearsAtCompany, YearsInCurrentRole, YearsSinceLastPromotion, YearsWithCurrManager
Example scaling applied (to df_scaled_example, not df). First 5 Age/MonthlyIncome:
| Age      | MonthlyIncome   |
|:---------|:----------------|
| 0.547619 | 0.262454        |
| 0.738095 | 0.217009        |
| 0.452381 | 0.0569247       |
| 0.357143 | 0.100053        |
| 0.214286 | 0.129489        |


In [ ]:
# --- Step 8: Save the Cleaned Dataset ---
print("\n--- Step 8: Save the Cleaned Dataset ---")
# The DataFrame 'df' now contains the cleaned data (after steps 1-6).
# If you applied scaling permanently in Step 7, 'df' would reflect that too.
cleaned_file_name = 'cleaned_WA_Fn-UseC_-HR-Employee-Attrition.csv'
print(f"Saving the cleaned data to '{cleaned_file_name}'...")

try:
    # Save the final DataFrame to a CSV file
    df.to_csv(cleaned_file_name, index=False)
    print(f"Cleaned dataset successfully saved.")
except Exception as e:
    print(f"An error occurred while saving the file: {e}")

print("\n--- Data Cleaning Process Completed ---")
# Display final info of the cleaned dataframe 'df'
print("\nFinal Cleaned Dataset Info:")
df.info()


--- Step 8: Save the Cleaned Dataset ---
Saving the cleaned data to 'cleaned_WA_Fn-UseC_-HR-Employee-Attrition.csv'...
Cleaned dataset successfully saved.

--- Data Cleaning Process Completed ---

Final Cleaned Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1470 entries, 0 to 1469
Data columns (total 46 columns):
 #   Column                             Non-Null Count  Dtype
---  ------                             --------------  -----
 0   Age                                1470 non-null   int64
 1   Attrition                          1470 non-null   int64
 2   DailyRate                          1470 non-null   int64
 3   DistanceFromHome                   1470 non-null   int64
 4   Education                          1470 non-null   int64
 5   EmployeeNumber                     1470 non-null   int64
 6   EnvironmentSatisfaction            1470 non-null   int64
 7   HourlyRate                         1470 non-null   int64
 8   JobInvolvement                     1470 no

In [ ]:
# Step 9: Check new cleanned data
# Define the path to the cleaned file
cleaned_file_path = 'cleaned_WA_Fn-UseC_-HR-Employee-Attrition.csv'

print(f"--- Verifying the cleaned dataset: '{cleaned_file_path}' ---")

try:
    # Load the cleaned dataset
    df_cleaned = pd.read_csv(cleaned_file_path)
    print(f"\nDataset '{cleaned_file_path}' loaded successfully.")

    # 1. Re-check for Missing Values
    print("\n--- 1. Checking for Missing Values ---")
    missing_values_cleaned = df_cleaned.isnull().sum()
    missing_values_cleaned = missing_values_cleaned[missing_values_cleaned > 0]
    if missing_values_cleaned.empty:
        print("Verification PASSED: No missing values found.")
    else:
        print("Verification FAILED: Missing values found in the cleaned file:")
        print(missing_values_cleaned.to_markdown(numalign="left", stralign="left"))

    # 2. Re-check for Duplicate Rows
    print("\n--- 2. Checking for Duplicate Rows ---")
    num_duplicates_cleaned = df_cleaned.duplicated().sum()
    if num_duplicates_cleaned == 0:
        print("Verification PASSED: No duplicate rows found.")
    else:
        print(f"Verification FAILED: Found {num_duplicates_cleaned} duplicate rows in the cleaned file.")

    # 3. Check if Single-Value Columns were Removed
    print("\n--- 3. Checking for Absence of Single-Value Columns ---")
    original_single_value_cols = ['EmployeeCount', 'Over18', 'StandardHours']
    cols_still_present = [col for col in original_single_value_cols if col in df_cleaned.columns]
    if not cols_still_present:
        print("Verification PASSED: Single-value columns ('EmployeeCount', 'Over18', 'StandardHours') are absent.")
    else:
        print(f"Verification FAILED: The following single-value columns are still present: {', '.join(cols_still_present)}")

    # 4. Check Data Types for Encoding
    print("\n--- 4. Checking Data Types for Encoding ---")
    object_columns = df_cleaned.select_dtypes(include='object').columns.tolist()
    # EmployeeNumber is expected to remain as int64, not object. Attrition should be int64 after mapping.
    # All other original object columns should now be numerical (int/uint8 from get_dummies).
    unexpected_object_cols = [col for col in object_columns if col != 'EmployeeNumber'] # Adjust if EmployeeNumber is expected as object

    if not unexpected_object_cols:
        print("Verification PASSED: No unexpected 'object' type columns found (indicating successful encoding).")
        # Optionally check if Attrition is now numeric
        if 'Attrition' in df_cleaned.columns and pd.api.types.is_numeric_dtype(df_cleaned['Attrition']):
             print("  - 'Attrition' column is correctly mapped to a numeric type.")
        elif 'Attrition' in df_cleaned.columns:
             print(f"  - Warning: 'Attrition' column is present but has type {df_cleaned['Attrition'].dtype}, expected numeric.")

    else:
        print(f"Verification FAILED: Found unexpected 'object' type columns: {', '.join(unexpected_object_cols)}")

    # 5. Check Final Shape and Columns (Optional)
    print("\n--- 5. Final Shape and Columns Overview ---")
    print(f"Shape of the cleaned dataset: {df_cleaned.shape}")
    print("Columns in the cleaned dataset:")
    print(df_cleaned.columns.tolist())
    print("\nCleaned Dataset Info:")
    df_cleaned.info()


except FileNotFoundError:
    print(f"Error: The cleaned file '{cleaned_file_path}' was not found. Please ensure Step 8 ran successfully.")
except Exception as e:
    print(f"An error occurred during verification: {e}")

--- Verifying the cleaned dataset: 'cleaned_WA_Fn-UseC_-HR-Employee-Attrition.csv' ---

Dataset 'cleaned_WA_Fn-UseC_-HR-Employee-Attrition.csv' loaded successfully.

--- 1. Checking for Missing Values ---
Verification PASSED: No missing values found.

--- 2. Checking for Duplicate Rows ---
Verification PASSED: No duplicate rows found.

--- 3. Checking for Absence of Single-Value Columns ---
Verification PASSED: Single-value columns ('EmployeeCount', 'Over18', 'StandardHours') are absent.

--- 4. Checking Data Types for Encoding ---
Verification PASSED: No unexpected 'object' type columns found (indicating successful encoding).
  - 'Attrition' column is correctly mapped to a numeric type.

--- 5. Final Shape and Columns Overview ---
Shape of the cleaned dataset: (1470, 46)
Columns in the cleaned dataset:
['Age', 'Attrition', 'DailyRate', 'DistanceFromHome', 'Education', 'EmployeeNumber', 'EnvironmentSatisfaction', 'HourlyRate', 'JobInvolvement', 'JobLevel', 'JobSatisfaction', 'MonthlyIn